<a href="https://colab.research.google.com/github/Akshat977/Applied-Data-Science-Capstone-Coursera-/blob/main/week%203%20final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

In [2]:
import requests
import urllib.request
import pandas as pd
import csv
from bs4 import BeautifulSoup

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
print(response.status_code)


200


In [5]:
soup = BeautifulSoup(response.text,"html.parser")
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"915dbb71-6634-4d20-83af-32352389c4d3","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":979555370,"wgRevisionId":979555370,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Ontario","

In [7]:
table = soup.findAll('table',{"class":"sortable"})[0]
values =[]
#There are 12 rows out of which we  need only rows from 1 to 11.
tr = table.findAll(['tr'])
tr

[<tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighbourhood
 </th></tr>, <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>, <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>, <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>, <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>, <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>, <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>, <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>, <tr>
 <td>M2B
 </td>
 <td>Not assigned
 </td>
 <td>Not a

In [8]:
csvFile = open("toronto.csv",'wt',newline='',encoding='utf-8')
writer = csv.writer(csvFile)  
try:   
        for cell in tr:
            th = cell.find_all('th')
            th_data = [col.text.strip('\n') for col in th]
            td = cell.find_all('td')
            row = [i.text.replace('\n','') for i in td]
            writer.writerow(th_data+row)      
        
finally:   
    csvFile.close()

In [9]:
df=pd.read_csv('toronto.csv')
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


**CLEANING THE DATAFRAME:**
   

*    Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
*   More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


*   If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


In [10]:
for index,row in df.iterrows():
  if row['Borough']=='Not assigned':
    df=df.drop(index)
  if (row['Borough']!='Not assigned' and row['Neighbourhood']=='Not assigned'):
    row['Neighbourhood']=row['Borough'] 
df.reset_index(inplace=True,drop=True)
df.head(10)     

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [11]:
df.shape

(103, 3)

In [16]:
coor=pd.read_csv('http://cocl.us/Geospatial_data')
coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df=pd.merge(df,coor,on='Postal Code')
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [18]:
df.shape

(103, 5)

In [19]:
#working only with toronto:
df.columns=['Postal Code','Borough','Neighborhood','Latitude','Longitude']
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [22]:
for index,row in df.iterrows():
  if 'Toronto' not in row['Borough']:
    df=df.drop(index)
df.reset_index(drop=True, inplace=True)
df.head()    

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [23]:
CLIENT_ID = 'FU3OXQ2S3TAJBGRJKVMSDN31IHBMUWIT1TQORGLQVV4YQFNA' # your Foursquare ID
CLIENT_SECRET = 'DARJDUYHI1QPIC45U5T12V33TOHYM4MTJONIOAFMVXVVYEPM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FU3OXQ2S3TAJBGRJKVMSDN31IHBMUWIT1TQORGLQVV4YQFNA
CLIENT_SECRET:DARJDUYHI1QPIC45U5T12V33TOHYM4MTJONIOAFMVXVVYEPM


In [24]:
#lets exploere only one borough first i.e. Downtown Toronto:
#and one neighbourhood: The beaches
lat_beach= 43.676357
lon_beach=-79.293031
address='Downtown Toronto, Canada'

In [46]:
radius=500
limit=50
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&Limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat_beach,lon_beach,radius,limit)


In [48]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f92d8563f31f819377dd3d0'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bd461bc77b29c74a07d9282-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d159941735',
         'name': 'Trail',
         'pluralName': 'Trails',
         'primary': True,
         'shortName': 'Trail'}],
       'id': '4bd461bc77b29c74a07d9282',
       'location': {'address': 'Glen Manor',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Queen St.',
        'distance': 89,
        'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682

In [34]:
from pandas.io.json import json_normalize
venue_beach=results['response']['groups'][0]['items']
venue_beach=json_normalize(venue_beach)
venue_beach.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.postalCode,venue.venuePage.id
0,e-0-4bd461bc77b29c74a07d9282-0,0,"[{'summary': 'This spot is popular', 'type': '...",4bd461bc77b29c74a07d9282,Glen Manor Ravine,Glen Manor,Queen St.,43.676821,-79.293942,"[{'label': 'display', 'lat': 43.67682094413784...",89,CA,Toronto,ON,Canada,"[Glen Manor (Queen St.), Toronto ON, Canada]","[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",0,[],NaN,NaN
1,e-0-4ad4c062f964a52011f820e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4ad4c062f964a52011f820e3,The Big Carrot Natural Food Market,125 Southwood Dr,NaN,43.678879,-79.297734,"[{'label': 'display', 'lat': 43.678879, 'lng':...",471,CA,Toronto,ON,Canada,"[125 Southwood Dr, Toronto ON M4E 0B8, Canada]","[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",0,[],M4E 0B8,75150878
2,e-0-4b8daea1f964a520480833e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b8daea1f964a520480833e3,Grover Pub and Grub,676 Kingston Rd.,at Main St.,43.679181,-79.297215,"[{'label': 'display', 'lat': 43.67918143494101...",460,CA,Toronto,ON,Canada,"[676 Kingston Rd. (at Main St.), Toronto ON M4...","[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",0,[],M4E 1R4,NaN
3,e-0-4df91c4bae60f95f82229ad5-3,0,"[{'summary': 'This spot is popular', 'type': '...",4df91c4bae60f95f82229ad5,Upper Beaches,NaN,NaN,43.680563,-79.292869,"[{'label': 'display', 'lat': 43.68056321147582...",468,CA,Toronto,ON,Canada,"[Toronto ON, Canada]","[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",0,[],NaN,NaN


In [60]:
for index,row in venue_beach.iterrows():
  print(row['venue.categories'][0]['name'])

Trail
Health Food Store
Pub
Neighborhood


**FIND VENUES FOR EACH BOROUGH: VERY IMPORTANT**

In [62]:
#now for each borough, for each neighborhood find all the venues:
#FIND VENUES FOR EACH BOROUGH::
#
k=list()
for index, row in df.iterrows():
  l=row['Neighborhood'].split(',')
  label=l[0]
  lat_cur=row['Latitude']
  lon_cur=row['Longitude']
  radius=500
  limit=100
  url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&Limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat_cur,lon_cur,radius,limit)
  results=requests.get(url).json()
  results=results['response']['groups'][0]['items']
  results=pd.json_normalize(results)
  for i,r in results.iterrows():
    if(r['venue.categories'][0]['name']=='Neighborhood'):
      continue
    j=list()
    j.append(row['Borough'])
    j.append(row['Postal Code'])
    j.append(label)
    j.append(row['Latitude'])
    j.append(row['Longitude'])
    j.append(r['venue.name'])
    j.append(r['venue.categories'][0]['name'])
    j.append(r['venue.location.lat'])
    j.append(r['venue.location.lng'])
    k.append(j)



In [63]:
q_df=pd.DataFrame(k)
q_df.columns=['Borough','Postal Code','Neighborhood','Neighborhood_Latitude','Neighborhood_Longitude','Venue_Name','Venue_category','Latitude','Longitude']
q_df.head()

,Borough,Postal Code,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue_Name,Venue_category,Latitude,Longitude
0,Downtown Toronto,M5A,Regent Park,43.65426,-79.360636,Roselle Desserts,Bakery,43.653447,-79.362017
1,Downtown Toronto,M5A,Regent Park,43.65426,-79.360636,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Downtown Toronto,M5A,Regent Park,43.65426,-79.360636,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Downtown Toronto,M5A,Regent Park,43.65426,-79.360636,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Downtown Toronto,M5A,Regent Park,43.65426,-79.360636,Impact Kitchen,Restaurant,43.656369,-79.356980


In [64]:
q_df.shape

(847, 9)

In [65]:
q_df.groupby('Neighborhood').count()

,Borough,Postal Code,Neighborhood_Latitude,Neighborhood_Longitude,Venue_Name,Venue_category,Latitude,Longitude
Neighborhood,,,,,,,,
Berczy Park,30,30,30,30,30,30,30,30
Brockton,23,23,23,23,23,23,23,23
Business reply mail Processing Centre,16,16,16,16,16,16,16,16
CN Tower,16,16,16,16,16,16,16,16
Central Bay Street,30,30,30,30,30,30,30,30
Christie,16,16,16,16,16,16,16,16
Church and Wellesley,30,30,30,30,30,30,30,30
Commerce Court,30,30,30,30,30,30,30,30
Davisville,30,30,30,30,30,30,30,30


In [66]:
print(len(q_df['Venue_category'].unique()))

193


**ANALYSE EACH NEIGHBORHOOD:**

In [70]:
neighbor=pd.DataFrame(q_df['Neighborhood'])
df_encode=pd.get_dummies(q_df['Venue_category'],prefix='',prefix_sep='')
df_encode=pd.concat([neighbor,df_encode],axis=1)
df_encode.head(10)

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,...,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [71]:
df_grouped=df_encode.groupby('Neighborhood').mean().reset_index()
df_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,...,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,0.0,0.033333,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.033333,0.0,0.000000,0.033333,0.066667,0.0,0.0,0.033333,0.0000,0.0,0.0000,0.033333,0.0000,0.000000,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0000,0.0000,0.033333,0.0,0.0,0.0,0.000000,0.0000,0.066667,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.033333,0.0,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000
1,Brockton,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.043478,0.0,0.043478,0.000000,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.0000,0.086957,0.0000,0.000000,0.0,0.043478,0.0,0.0000,0.130435,0.0,0.0,...,0.0,0.000000,0.0,0.0000,0.0000,0.043478,0.0,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.043478,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,Business reply mail Processing Centre,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0625,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.0000,0.000000,0.0625,0.000000,0.0,0.062500,0.0,0.0625,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0625,0.0000,0.062500,0.0,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.0625,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,CN Tower,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.062500,0.000000,0.000000,0.0,0.0,0.000000,0.0625,0.0,0.0625,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0000,0.0625,0.000000,0.0,0.0,0.0,0.000000,0.0625,0.000000,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,0.0,0.000000,0.033333,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.0000,0.000000,0.0000,0.033333,0.0,0.000000,0.0,0.0000,0.066667,0.0,0.0,...,0.0,0.033333,0.0,0.0000,0.0000,0.000000,0.0,0.0,0.0,0.033333,0.0000,0.033333,0.0,0.0000,0.0,0.0,0.0,0.033333,0.0,0.0,0.000000,0.0,0.0,0.0,0.033333,0.0,0.000000,0.0,0.033333,0.000000,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.033333


In [72]:
df_grouped.shape

(38, 194)

In [85]:
# PRINTING THE TOP 5 MOST COMMON VENUES IN EACH NEIGHBORHOOD:
for index,row in df_grouped.iterrows():
  print('---------',row['Neighborhood'],'-------------')
  t=df_grouped[df_grouped['Neighborhood']==row['Neighborhood']].T.reset_index()
  t.drop(0,inplace=True)
  t.columns=['venue','freq']
  t['freq']=t['freq'].astype(float)
  t=t.round({'freq':4})
  t=t.sort_values('freq')
  print(t.head(5))

--------- Berczy Park -------------
                  venue  freq
1               Airport   0.0
117  Italian Restaurant   0.0
120       Jewelry Store   0.0
121           Juice Bar   0.0
122   Korean Restaurant   0.0
--------- Brockton -------------
                         venue  freq
1                      Airport   0.0
122          Korean Restaurant   0.0
123                       Lake   0.0
124  Latin American Restaurant   0.0
125         Light Rail Station   0.0
--------- Business reply mail Processing Centre -------------
                 venue  freq
1              Airport   0.0
120      Jewelry Store   0.0
121          Juice Bar   0.0
122  Korean Restaurant   0.0
123               Lake   0.0
--------- CN Tower -------------
                         venue  freq
97       General Entertainment   0.0
122          Korean Restaurant   0.0
123                       Lake   0.0
124  Latin American Restaurant   0.0
125         Light Rail Station   0.0
--------- Central Bay Street ---------

In [95]:
#MAKE A DATAFRAME FROM THE 5 MOST COMMON VENUES:
venues_sorted=list()
col=['1st','2nd','3rd','4th','5th','6th','7th','8th','9th','10th']
for index,row in df_grouped.iterrows():
  k,x=list(),list()
  t=df_grouped[df_grouped['Neighborhood']==row['Neighborhood']].T.reset_index()
  t.drop(0,inplace=True)
  t.columns=['venue','freq']
  t['freq']=t['freq'].astype(float)
  t=t.round({'freq':4})
  t=t.sort_values('freq')
  for i in t['venue']:
    x.append(i)
    
  venues_sorted.append(x[0:10])
print(venues_sorted)  

[['Airport', 'Italian Restaurant', 'Jewelry Store', 'Juice Bar', 'Korean Restaurant', 'Lake', 'Latin American Restaurant', 'Light Rail Station', 'Lounge', 'Malay Restaurant'], ['Airport', 'Korean Restaurant', 'Lake', 'Latin American Restaurant', 'Light Rail Station', 'Liquor Store', 'Lounge', 'Malay Restaurant', 'Market', 'Martial Arts School'], ['Airport', 'Jewelry Store', 'Juice Bar', 'Korean Restaurant', 'Lake', 'Latin American Restaurant', 'Liquor Store', 'Lounge', 'Malay Restaurant', 'Market'], ['General Entertainment', 'Korean Restaurant', 'Lake', 'Latin American Restaurant', 'Light Rail Station', 'Liquor Store', 'Lounge', 'Malay Restaurant', 'Market', 'Martial Arts School'], ['Airport', 'Indian Restaurant', 'Indie Movie Theater', 'Intersection', 'Jazz Club', 'Jewelry Store', 'Juice Bar', 'Korean Restaurant', 'Lake', 'Ice Cream Shop'], ['Airport', 'Lake', 'Latin American Restaurant', 'Light Rail Station', 'Liquor Store', 'Lounge', 'Malay Restaurant', 'Market', 'Martial Arts Schoo

In [97]:
venues_sorted=pd.DataFrame(venues_sorted)
venues_sorted.columns=col
venues_sorted=pd.concat([df_grouped['Neighborhood'],venues_sorted],axis=1)
venues_sorted.head()

,Neighborhood,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,Berczy Park,Airport,Italian Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lounge,Malay Restaurant
1,Brockton,Airport,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School
2,Business reply mail Processing Centre,Airport,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Malay Restaurant,Market
3,CN Tower,General Entertainment,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School
4,Central Bay Street,Airport,Indian Restaurant,Indie Movie Theater,Intersection,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Ice Cream Shop


In [98]:
venues_sorted.shape

(38, 11)

**CLUSTER NEIHBORHOODS**

In [100]:
from sklearn.cluster import KMeans
model=KMeans(n_clusters=7,init='k-means++',n_init=13)
df_cluster=df_grouped.drop('Neighborhood',1)
model.fit(df_cluster)
print(model.labels_[0:10])

[5 0 5 5 5 0 5 0 5 5]


In [101]:
venues_sorted.shape

(38, 11)

In [125]:
#venues_sorted.insert(0,'Cluster labels',model.labels_)
venues_sorted.head()

,Cluster labels,Neighborhood,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,5,Berczy Park,Airport,Italian Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lounge,Malay Restaurant
1,0,Brockton,Airport,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School
2,5,Business reply mail Processing Centre,Airport,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Malay Restaurant,Market
3,5,CN Tower,General Entertainment,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School
4,5,Central Bay Street,Airport,Indian Restaurant,Indie Movie Theater,Intersection,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Ice Cream Shop


In [107]:
q_df.head()

,Borough,Postal Code,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue_Name,Venue_category,Latitude,Longitude
0,Downtown Toronto,M5A,Regent Park,43.65426,-79.360636,Roselle Desserts,Bakery,43.653447,-79.362017
1,Downtown Toronto,M5A,Regent Park,43.65426,-79.360636,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Downtown Toronto,M5A,Regent Park,43.65426,-79.360636,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Downtown Toronto,M5A,Regent Park,43.65426,-79.360636,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Downtown Toronto,M5A,Regent Park,43.65426,-79.360636,Impact Kitchen,Restaurant,43.656369,-79.356980


In [115]:
q_df.drop_duplicates(inplace=True,keep='first',subset='Neighborhood')
q_df.head()

,Borough,Postal Code,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue_Name,Venue_category,Latitude,Longitude
0,Downtown Toronto,M5A,Regent Park,43.654260,-79.360636,Roselle Desserts,Bakery,43.653447,-79.362017
30,Downtown Toronto,M7A,Queen's Park,43.662301,-79.389494,Queen's Park,Park,43.663946,-79.392180
60,Downtown Toronto,M5B,Garden District,43.657162,-79.378937,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
90,Downtown Toronto,M5C,St. James Town,43.651494,-79.375418,Gyu-Kaku Japanese BBQ,Japanese Restaurant,43.651422,-79.375047
120,East Toronto,M4E,The Beaches,43.676357,-79.293031,Glen Manor Ravine,Trail,43.676821,-79.293942


In [127]:
final_df=pd.merge(venues_sorted,q_df,on='Neighborhood')
final_df=final_df.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,18,19]]
final_df.head()

,Cluster labels,Neighborhood,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th,Borough,Postal Code,Latitude,Longitude
0,5,Berczy Park,Airport,Italian Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lounge,Malay Restaurant,Downtown Toronto,M5E,43.646712,-79.374768
1,0,Brockton,Airport,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,West Toronto,M6K,43.637036,-79.424802
2,5,Business reply mail Processing Centre,Airport,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,East Toronto,M7Y,43.663483,-79.319824
3,5,CN Tower,General Entertainment,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Downtown Toronto,M5V,43.631683,-79.396033
4,5,Central Bay Street,Airport,Indian Restaurant,Indie Movie Theater,Intersection,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Ice Cream Shop,Downtown Toronto,M5G,43.658421,-79.385613


In [128]:
final_df.shape

(38, 16)

In [130]:
address='Toronto, Canada'
geolocator=Nominatim(user_agent='foursquare_agent')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print(latitude,longitude)

43.6534817 -79.3839347


**VISUALIZING THE FINAL DATA:**

In [131]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# set color scheme for the clusters
x = np.arange(7)
ys = [i + x + (i*x)**2 for i in range(7)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [132]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_df['Latitude'], final_df['Longitude'], final_df['Neighborhood'], final_df['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**EXAMINING EACH CLUSTER:**

In [135]:
for i in range(7):
  print('---------------','CLUSTER',i,'--------------')
  for index,row in final_df.iterrows():
    if row['Cluster labels']==i:
      print(final_df.iloc[index])

--------------- CLUSTER 0 --------------
Cluster labels                            0
Neighborhood                       Brockton
1st                                 Airport
2nd                       Korean Restaurant
3rd                                    Lake
4th               Latin American Restaurant
5th                      Light Rail Station
6th                            Liquor Store
7th                                  Lounge
8th                        Malay Restaurant
9th                                  Market
10th                    Martial Arts School
Borough                        West Toronto
Postal Code                             M6K
Latitude                             43.637
Longitude                          -79.4248
Name: 1, dtype: object
Cluster labels                            0
Neighborhood                       Christie
1st                                 Airport
2nd                                    Lake
3rd               Latin American Restaurant
4th         